# Metric Scoping

Reference data from [here](https://data.europa.eu/euodp/data/dataset/cordisref-data).

- [ ] Number of citations
- [ ] Types of publication outputs
- [ ] Access rights
- [ ] Did another collaboration happen afterwards?


- [ ] Public private balance
- [ ] Country diversity
- [ ] Org diversity

## Preamble

In [ ]:
%run notebook_preamble.ipy

pd.set_option('max_columns', 99)

In [ ]:
import seaborn as sns
from itertools import chain
from collections import Counter, defaultdict
import pickle
import ast
from datetime import datetime

from eu_funding.visualization.visualize import pdf_cdf
import seaborn as sns

sns.set_context('notebook')
# from src.visualization.visualize import pdf_cdf

## Data

### Projects

Each funding programme has a projects dataset, which contain research projects funded by EC programmes.
- `rcn`
- `id`
- `acronym`
- `status`
- `programme`
- `topics`
- `frameworkProgramme`
- `title`
- `startDate`
- `endDate`
- `projectUrl`
- `objective`
- `totalCost`
- `ecMaxContribution`
- `call`
- `fundingScheme`
- `coordinator`
- `coordinatorCountry`
- `participants`
- `participantCountries`
- `subjects`

### H2020

In [ ]:
h2020_projects_df = pd.read_csv(
    os.path.join(cordis_h2020_path, 'cordis-h2020projects.csv'),
    sep=';',
    encoding='iso-8859-1',
    parse_dates=['startDate', 'endDate'],
    infer_datetime_format=True,
    decimal=','
)
h2020_orgs_df = pd.read_csv(
    os.path.join(cordis_h2020_path, 'cordis-h2020organizations.csv'),
    sep=';',
    encoding='iso-8859-1',
    decimal=',',
)
h2020_reports_df = pd.read_csv(
    os.path.join(cordis_h2020_path, 'cordis-h2020reports.csv'),
)
h2020_pi_df = pd.read_excel(
    os.path.join(cordis_h2020_path, 'cordis-h2020-erc-pi.xlsx'),
    sheet_name='PI'
)
h2020_fellows_r1_df = pd.read_excel(
    os.path.join(cordis_h2020_path, 'cordis-h2020-msca-fellows.xls'),
    sheet_name='Report 1',
    header=3
).set_index('Project Number')
h2020_fellows_r2_df = pd.read_excel(
    os.path.join(cordis_h2020_path, 'cordis-h2020-msca-fellows.xls'),
    sheet_name='Report 2'
).set_index('projectId')
h2020_fellows_df = h2020_fellows_r1_df.join(h2020_fellows_r2_df, how='outer').reset_index().rename(columns={'index': 'projectId'})

In [ ]:
h2020_projects_df.head(1)

In [ ]:
h2020_projects_df.shape

In [ ]:
h2020_reports_df.head(1)

In [ ]:
h2020_reports_df.shape

In [ ]:
h2020_orgs_df.head(1)

In [ ]:
h2020_orgs_df.shape

In [ ]:
h2020_pi_df.head(1)

In [ ]:
h2020_fellows_df.head(1)

### FP7

In [ ]:
fp7_projects_df = pd.read_csv(
    os.path.join(cordis_fp7_path, 'cordis-fp7projects.csv'),
    sep=';',
    encoding='iso-8859-1',
    parse_dates=['startDate', 'endDate'],
    infer_datetime_format=True,
    decimal=','
)
fp7_orgs_df = pd.read_csv(
    os.path.join(cordis_fp7_path, 'cordis-fp7organizations.csv'),
    sep=';',
    encoding='iso-8859-1',
    decimal=',',
)
fp7_reports_df = pd.read_csv(
    os.path.join(cordis_fp7_path, 'cordis-fp7reports.csv'),
).drop('Unnamed: 16', axis=1)

In [ ]:
fp7_projects_df.head(1)

In [ ]:
fp7_orgs_df.head(1)

In [ ]:
fp7_projects_df[fp7_projects_df['id'] == 240271]

### Combining Programme Datasets

In [ ]:
cordis_projects_df = pd.concat([h2020_projects_df, fp7_projects_df])
cordis_reports_df = pd.concat([h2020_reports_df, fp7_reports_df])
cordis_orgs_df = pd.concat([h2020_orgs_df, fp7_orgs_df])

In [ ]:
def all_particpants(coordinator, participants):
    if pd.isnull(participants):
        return coordinator
    else:
        if coordinator in participants:
            return participants
        else:
            return participants + ';' + coordinator

cordis_projects_df['allCountries'] = cordis_projects_df.apply(
    lambda x: all_particpants(x['coordinatorCountry'], x['participantCountries']),
    axis=1
).fillna('NA')
cordis_projects_df['countries_count'] = [len(s.split(';')) for s in cordis_projects_df['allCountries']]

cordis_projects_df['allParticipants'] = cordis_projects_df.apply(
    lambda x: all_particpants(x['coordinator'], x['participants']),
    axis=1
)
cordis_projects_df['participant_count'] = [len(s.split(';')) for s in cordis_projects_df['allParticipants']]

### Reference Data

#### Activity Types

In [ ]:
cordis_activity_ref_df = pd.read_csv(
    os.path.join(cordis_ref_path, 'cordisref-organizationActivityType.csv'),
    sep=';'
)

In [ ]:
cordis_activity_ref_df

#### Countries

In [ ]:
cordis_countries_ref_df = pd.read_csv(
    os.path.join(cordis_ref_path, 'cordisref-countries.csv'),
    sep=';'
)

In [ ]:
cordis_countries_ref_df.head(10)

#### Topics

In [ ]:
cordis_topics_ref_df = pd.read_csv(
    os.path.join(cordis_ref_path, 'cordisref-H2020topics.csv'),
    sep=';'
)

In [ ]:
cordis_topics_ref_df.head()

#### H2020 Programmes

In [ ]:
cordis_h2020_programmes_ref_df = pd.read_csv(
    os.path.join(cordis_ref_path, 'cordisref-H2020programmes.csv'),
#     sep=';'
)

In [ ]:
cordis_h2020_programmes_ref_df.head()

#### Funding Schemes

In [ ]:
cordis_funding_schemes_ref_df = pd.read_csv(
    os.path.join(cordis_ref_path, 'cordisref-projectFundingSchemeCategory.csv'),
    sep=';'
)

In [ ]:
cordis_funding_schemes_ref_df.head()

#### SIC Codes

In [ ]:
cordis_sic_codes_ref_df = pd.read_csv(
    os.path.join(cordis_ref_path, 'cordisref-sicCode.csv'),
    sep=';'
)

In [ ]:
cordis_sic_codes_ref_df[cordis_sic_codes_ref_df['language'] == 'en'].head()

### OpenAIRE

In [ ]:
list_cols = ['ec_project_codes', 'categories', 'children', 'source', 'subjects']

openaire_publications_df = pd.read_csv(
    os.path.join(inter_data_path, 'openaire_publications_20190702.csv'),
    converters={k: ast.literal_eval for k in list_cols}
)


In [ ]:
openaire_publications_df.drop_duplicates(['title'], inplace=True)

In [ ]:
openaire_publications_df['pub_id'] = openaire_publications_df.index

In [ ]:
openaire_publications_df['n_projects'] = [len(s) for s in openaire_publications_df['ec_project_codes']]

In [ ]:
expanded_records = []
for record in openaire_publications_df.to_dict(orient='records'):
    ec_project_codes = record.pop('ec_project_codes')
    for ec_code in ec_project_codes:
        new_record = record.copy()
        new_record['ec_project_code'] = int(ec_code)
        expanded_records.append(new_record)

In [ ]:
oa_pubs_df = pd.DataFrame().from_records(expanded_records)

In [ ]:
oa_pubs_df.shape

### OpenAIRE Children

In [ ]:
child_records = []
for i, record in zip(oa_pubs_df.index, oa_pubs_df.to_dict(orient='records')):
    children = record['children']
    for child in children:
        child['ec_project_code'] = record['ec_project_code']
        child['pub_id'] = record['pub_id']
        child['oai'] = i
        child_records.append(child)

In [ ]:
oa_child_df = pd.DataFrame().from_records(child_records)

In [ ]:
oa_child_df.shape

In [ ]:
oa_child_df.drop_duplicates(['name', 'ec_project_code', 'collectedfrom', 'dateofacceptance'], inplace=True)

In [ ]:
oa_child_df.head()

### CrossRef Works

In [ ]:
crossref_works_records = []

for file in os.listdir(crossref_works_path):
    if '.txt' in file:
        with open(os.path.join(crossref_works_path, file), 'r') as f:
            crossref_works_records.extend(json.load(f))

In [ ]:
crossref_works_records = [c for c in crossref_works_records if type(c) == dict]

In [ ]:
crossref_works_df = pd.DataFrame().from_records(crossref_works_records)

In [ ]:
def parse_date_parts(date_dict):
    date_parts = date_dict.get('date-parts')
    if date_parts is not None:
        date_parts = date_parts[0]
        if date_parts[0] is not None:
            fill = 3 - len(date_parts)
            for _ in range(fill):
                date_parts.append(1)
            dt = datetime(*date_parts)
            return dt
    return None

In [ ]:
crossref_works_df['issued_dt'] = crossref_works_df['issued'].apply(lambda x: parse_date_parts(x))

In [ ]:
collected_date = datetime(2019, 2, 1)

In [ ]:
crossref_works_df['age'] = collected_date - crossref_works_df['issued_dt']

In [ ]:
crossref_works_df['age_years'] = crossref_works_df['age'].dt.days / 365.25

In [ ]:
crossref_works_df = crossref_works_df[crossref_works_df['age_years'] <= 20]

In [ ]:
crossref_works_df.head(1)

### Joining OpenAIRE + CrossRef

In [ ]:
crossref_works_df.shape

In [ ]:
oa_cr_pubs_df = oa_pubs_df.merge(crossref_works_df, left_on='doi', right_on='DOI')

In [ ]:
oa_cr_pubs_df.head(3)

In [ ]:
oa_cr_pubs_df.to_csv(os.path.join(inter_data_path, 'openaire_crossref_publications.csv'), index=False)

## EDA

### Citations

In [ ]:
oa_cr_pubs_df['project_pub_contribution'] = 1 / oa_cr_pubs_df['n_projects']
oa_cr_pubs_df['normalised_citation_count'] = (oa_cr_pubs_df['is-referenced-by-count'] *
                                              oa_cr_pubs_df['project_pub_contribution'])

In [ ]:
project_publications = oa_cr_pubs_df[[
    'ec_project_code', 'normalised_citation_count', 'project_pub_contribution']
].groupby(
    'ec_project_code'
).sum()

In [ ]:
cordis_projects_df = cordis_projects_df.merge(
    project_publications.reset_index(),
    left_on='id',
    right_on='ec_project_code',
    how='left'
)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(12, 4))
ax[0].hist(cordis_projects_df['normalised_citation_count'], bins=100)
ax[1].hexbin(
    np.log10(cordis_projects_df['ecMaxContribution']),
    np.log10(cordis_projects_df['normalised_citation_count']),
    bins='log',
    mincnt=1
)
plt.show()

In [ ]:
n_publications = oa_cr_pubs_df.groupby('ec_project_code')['doi'].count()

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(12, 4))

n_publications = oa_cr_pubs_df.groupby('ec_project_code')['doi'].count()
bins = [-1, 5, 20, 50, 200, 500]
n_publications = pd.cut(
    n_publications, 
    bins=bins,
    include_lowest=True
)
n_publications.value_counts(sort=False).plot.bar(rot=0, color="C0", ax=ax[0])
ax[0].set_xticklabels(['{} - {}'.format(l+1, r) for l, r in zip(bins[:-1], bins[1:])])

bins = [-1, 10, 100, 1000, 10000]
citations = pd.cut(
    oa_cr_pubs_df['is-referenced-by-count'], 
    bins=bins,
    include_lowest=True
)
citations.value_counts(sort=False).plot.bar(rot=0, color="C0", ax=ax[1])
ax[1].set_xticklabels(['{} - {}'.format(l+1, r) for l, r in zip(bins[:-1], bins[1:])])

ax[0].set_xlabel('N Publications per Project')
ax[0].set_ylabel('Frequency')
ax[1].set_xlabel('N Citations per Publication')
ax[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

### N Publications

In [ ]:
end = 30
fig, ax = plt.subplots()
cordis_proj_participant_count_group = cordis_projects_df.groupby('participant_count')
n_particpant_pub_med = cordis_proj_participant_count_group['project_pub_contribution'].median()[:end]
n_particpant_pub_lq = cordis_proj_participant_count_group['project_pub_contribution'].quantile(0.25)[:end]
n_particpant_pub_uq = cordis_proj_participant_count_group['project_pub_contribution'].quantile(0.75)[:end]
ax.plot(n_particpant_pub_med, linewidth=3, color='C7')
ax.plot(n_particpant_pub_lq, color='C3')
ax.plot(n_particpant_pub_uq, color='C2')

### Access Type

In [ ]:
oa_child_df_access_types = pd.get_dummies(oa_child_df['access'])

In [ ]:
oa_child_df = oa_child_df.join(pd.get_dummies(oa_child_df['access']))

In [ ]:
proj_access_counts = oa_child_df.groupby(
    'ec_project_code'
)['Closed Access', 'Embargo', 'Open Access', 'Restricted', 'UNKNOWN'].sum()

In [ ]:
pubs_sum = proj_access_counts.sum(axis=1)

In [ ]:
# calculate the normalised distribution of access types for each project
access_types = ['Closed Access', 'Embargo', 'Open Access', 'UNKNOWN', 'Restricted']

for a_t in access_types:
    proj_access_counts[a_t] = proj_access_counts[a_t] / pubs_sum

In [ ]:
cordis_orgs_df['activityType'].fillna('OTH', inplace=True)

In [ ]:
cordis_activity_types_ohe = pd.get_dummies(cordis_orgs_df['activityType'])

In [ ]:
cordis_activity_types_ohe['projectID'] = cordis_orgs_df['projectID']

In [ ]:
cordis_proj_activity_dist = cordis_activity_types_ohe.groupby('projectID').sum()
cordis_proj_total_orgs = cordis_proj_activity_dist.sum(axis=1)
for a_t in cordis_activity_ref_df['Code']:
    cordis_proj_activity_dist[a_t] = cordis_proj_activity_dist[a_t] / cordis_proj_total_orgs

In [ ]:
access_org_type_df = cordis_proj_activity_dist.merge(
    proj_access_counts, 
    left_index=True, 
    right_index=True,
    how='inner'
)

In [ ]:
fig, ax = plt.subplots(nrows=len(cordis_activity_ref_df['Code']), ncols=len(access_types), figsize=(16, 12))

for i, activity in enumerate(cordis_activity_ref_df['Code']):
    for j, access in enumerate(access_types):
        ax[i][j].hexbin(
            access_org_type_df[access],
            access_org_type_df[activity],
#             alpha=0.1,
            bins='log',
            mincnt=1
        )
        ax[i][j].set_xlabel(activity)
        ax[i][j].set_ylabel(access)
plt.tight_layout()
plt.show()

In [ ]:
cordis_activity_ref_df

In [ ]:
for activity in cordis_activity_ref_df['Code']:
    access_org_type_df['has_{}'.format(activity)] =  access_org_type_df[activity] > 0

In [ ]:
for access in access_types:
    access_org_type_df['has_{}'.format(access)] =  access_org_type_df[access] > 0

In [ ]:
fig, ax = plt.subplots(nrows=5, figsize=(6, 8))

for i, access in enumerate(['{}'.format(a) for a in access_types]):
    pdf_cdf(
        ax[i],
        access_org_type_df[access],
        bins=20
    )
    ax[i].set_xlabel(f'% of {access} Outputs')
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(16,4))
ax[0].hist(
    access_org_type_df['has_Open Access'][
        (access_org_type_df['has_PRC'] == True)],
)
ax[1].hist(
    access_org_type_df['has_Open Access'][
        (access_org_type_df['has_PRC'] == False) 
#         & (access_org_type_df['has_PUB'] == True)
    ],
)
ax[2].hist(
    access_org_type_df['has_Open Access'][
        (access_org_type_df['has_PRC'] == False) &
        (access_org_type_df['has_PUB'] == True)],
#     density='cumulative'
)
plt.show()

In [ ]:
access_org_type_df[
    (access_org_type_df['has_PRC'] == True) & 
    (access_org_type_df['has_HES'] == False)]['has_Closed Access'].mean()

In [ ]:
access_org_type_df.groupby('has_PRC')[['{}'.format(a) for a in access_types]].mean()

### Publication Type

In [ ]:
oa_child_df['type'].fillna('Article', inplace=True)

In [ ]:
cordis_project_codes = sorted(cordis_projects_df['id'])

In [ ]:
oa_child_df['in_cordis'] = oa_child_df['ec_project_code'].apply(lambda x: x in cordis_project_codes)

In [ ]:
fig, ax = plt.subplots()

oa_child_df['type'].value_counts().plot(kind='barh', color='C0', ax=ax)
ax.invert_yaxis()

In [ ]:
fig, ax = plt.subplots()

oa_child_df['type'][oa_child_df['in_cordis'] == True].value_counts().plot(kind='barh', color='C0', ax=ax)
ax.invert_yaxis()

In [ ]:
pub_types = oa_child_df['type'].unique()

In [ ]:
oa_child_df_pub_types = pd.get_dummies(oa_child_df['type'])
oa_child_df_pub_types['ec_project_code'] = oa_child_df['ec_project_code']

In [ ]:
proj_pub_type_counts = oa_child_df_pub_types.groupby(
    'ec_project_code'
)[pub_types].sum()

In [ ]:
pubs_sum = proj_access_counts.sum(axis=1)

In [ ]:
for p_t in pub_types:
    proj_pub_type_counts[f'{p_t}_normed'] = proj_pub_type_counts[p_t] / pubs_sum
    proj_pub_type_counts[f'{p_t}_normed'][np.isinf(proj_pub_type_counts[f'{p_t}_normed'])] = 0

In [ ]:
proj_pub_type_counts.reset_index(inplace=True)

In [ ]:
with open(os.path.join(model_path, 'gtr_text_models.p'), 'rb') as f:
    gtr_discipline_model = pickle.load(f)

In [ ]:
abstract_vecs = gtr_discipline_model[0].transform(cordis_projects_df['objective'])

In [ ]:
discipline_labels = gtr_discipline_model[1].predict(abstract_vecs)

In [ ]:
category_name_lookup = {6:'biological_sciences',
                        3:'physics',
                        0:'engineering_technology',
                        1:'environmental_sciences',
                        2:'social_sciences',
                        4:'arts_humanities',
                        5:'mathematics_computing',
                        7: 'medical_sciences'}

In [ ]:
cordis_projects_disciplines_df = pd.DataFrame(discipline_labels).rename(columns=category_name_lookup)

In [ ]:
cordis_projects_df.reset_index(inplace=True)
cordis_projects_df.drop('index', inplace=True, axis=1)

In [ ]:
cordis_project_publications_df = cordis_projects_df.join(
    cordis_projects_disciplines_df).merge(
    proj_pub_type_counts, 
    left_on='id', 
    right_on='ec_project_code',
    how='inner'
)

In [ ]:
cordis_project_publications_df

Groupby discipline. Calculate sum of each output type. Put in table.

In [ ]:
discipline_pub_type_counts = []

for _, discipline in category_name_lookup.items():
    x = cordis_project_publications_df.groupby(discipline)[pub_types].sum().to_dict(orient='records')[1]
    x['discipline'] = discipline
    discipline_pub_type_counts.append(x)

In [ ]:
disci_pub_type_df = pd.DataFrame().from_records(discipline_pub_type_counts).set_index('discipline')

In [ ]:
fig, ax = plt.subplots()
disci_pub_type_df.plot(kind='barh', stacked=True, ax=ax, colormap='tab20')
ax.legend(loc='right', bbox_to_anchor=(2.5, .5), ncol=2, 
            borderaxespad=0, frameon=False)
ax.set_xlabel('Frequency')
plt.show()

In [ ]:
cordis_project_publications_df.head()

### Lexical Diversity

### Collaborations

In [ ]:
authors = oa_pubs_df['authors'].str.split(';')

In [ ]:
c_authors = Counter(chain(*[a for a in authors if type(a) == list]))

In [ ]:
authors_h = [tuple(a) for a in authors if type(a) == list]

In [ ]:
c_authors = Counter(authors_h)

In [ ]:
c_authors.most_common(1000)

### Extra Stuff

In [ ]:
project_counts = [len(s) for s in openaire_publications_df['ec_project_codes']]
project_counts_c = Counter(project_counts)
n_codes, n_codes_count = zip(*project_counts_c.items())

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
ax.bar(n_codes, n_codes_count)
ax.set_xlabel('N EC Projects')
ax.set_ylabel('Frequency')
plt.show()

Around 50% of projects have no EC project associated. However, over 120,000 of them have at least one. Over 10,000 are attributed to two projects, while a smaller handful are associated with 3 or more.

In [ ]:
def flatten(nested):
    return list(chain(*nested))

In [ ]:
publication_types = flatten(openaire_publications_df['child_types'])
c_types = Counter(publication_types)
pub_type, pub_type_count = zip(*c_types.items())

fig, ax = plt.subplots(figsize=(8,4))
ax.barh(pub_type, pub_type_count)
ax.set_xlabel('Publication Type')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
access_types = flatten(openaire_publications_df['child_access'])
c_access = Counter(access_types)
access_type, access_type_count = zip(*c_access.items())

fig, ax = plt.subplots(figsize=(8,4))
ax.barh(access_type, access_type_count)
ax.set_xlabel('Child Access Type')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
accessright_counts = openaire_publications_df['bestaccessright'].value_counts()
fig, ax = plt.subplots(figsize=(8,4))
ax.barh(accessright_counts.index, accessright_counts.values)
ax.set_xlabel('Publication Access Type')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
accessright_counts = openaire_publications_df['publication_type'].value_counts()
fig, ax = plt.subplots(figsize=(8,4))
ax.barh(accessright_counts.index, accessright_counts.values)
ax.set_xlabel('Publication Access Type')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
langauge_counts = openaire_publications_df['language'].value_counts()
fig, ax = plt.subplots(figsize=(8,8))
ax.barh(langauge_counts.index, langauge_counts.values)
ax.set_xlabel('Language')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
category_c = Counter(flatten(openaire_publications_df['categories']))

In [ ]:
category_c.most_common(40)

### CrossRef Works

In [ ]:
crossref_works_records = []

for file in os.listdir(crossref_works_path):
    if '.txt' in file:
        with open(os.path.join(crossref_works_path, file), 'r') as f:
            crossref_works_records.extend(json.load(f))

In [ ]:
crossref_works_records = [c for c in crossref_works_records if type(c) == dict]

In [ ]:
crossref_works_df = pd.DataFrame().from_records(crossref_works_records)

In [ ]:
def parse_date_parts(date_dict):
    date_parts = date_dict.get('date-parts')
    if date_parts is not None:
        date_parts = date_parts[0]
        if date_parts[0] is not None:
            fill = 3 - len(date_parts)
            for _ in range(fill):
                date_parts.append(1)
            dt = datetime(*date_parts)
            return dt
    return None

In [ ]:
crossref_works_df['issued_dt'] = crossref_works_df['issued'].apply(lambda x: parse_date_parts(x))

In [ ]:
collected_date = datetime(2019, 2, 1)

In [ ]:
crossref_works_df['age'] = collected_date - crossref_works_df['issued_dt']

In [ ]:
crossref_works_df['age_years'] = crossref_works_df['age'].dt.days / 365.25

In [ ]:
crossref_works_df = crossref_works_df[crossref_works_df['age_years'] <= 20]

In [ ]:
crossref_works_df['type'].value_counts()

In [ ]:
plt.hist(crossref_works_df['age_years'], bins=100)
plt.show()

In [ ]:
ref_per_year = crossref_works_df['is-referenced-by-count'] / np.power(crossref_works_df['age_years'], 2)
# ref_per_year_log = np.log10(ref_per_year)
# ref_per_year_log = ref_per_year_log[~np.isinf(ref_per_year_log)]

plt.hist(ref_per_year, bins=1000)
# plt.xlim((0, 50))
plt.show()

In [ ]:
plt.hexbin(
    crossref_works_df['age_years'],
    np.log(crossref_works_df['is-referenced-by-count']),
#     bins='log',
    gridsize=40,
)

In [ ]:
crossref_works_df.head()

In [ ]:
crossref_works_df['is-referenced-by-count'].describe()

In [ ]:
plt.hist(crossref_works_df['is-referenced-by-count'], bins=100)
plt.show

### Lexical Diversity

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import ast

In [ ]:
cv = CountVectorizer(dtype=np.int32)

In [ ]:
from skbio.diversity.alpha import shannon

In [ ]:
sum(pd.isnull(openaire_publications_df['language']))

In [ ]:
entropies = []
lengths = []
n_papers = []

for pid in cordis_projects_df['id']:
    papers = openaire_publications_df[[True if str(pid) in s else False 
                                       for s in openaire_publications_df['ec_project_codes']]]
    papers = papers[~pd.isnull(papers['abstract'])]
    try:
        main_lang = papers['language'].value_counts().index[0]
        papers = papers[papers['language'] == main_lang]
        paper_count = len(papers)
    except:
        paper_count = len(papers)
    n_papers.append(paper_count)
    if paper_count > 0:
        mega_abstract = [' '.join([s for s in papers])]
        article_bow = cv.fit_transform(mega_abstract)
        for bow in article_bow:
            x = np.squeeze(np.asarray(bow.todense()))
            x = x[x != 0]
            entropy = shannon(x)
            lengths.append(bow.sum())
            entropies.append(entropy)
        
    else:
        entropies.append(0)
        lengths.append(0)


In [ ]:
papers

In [ ]:
fig, ax = plt.subplots()
ax.scatter(n_papers, entropies, alpha=0.05)
# ax.set_xscale('log')

In [ ]:
def func(x, a, b, c):
    return a * np.exp(-b * x) + c

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
import operator

In [ ]:
n_papers, entropies = zip(*sorted(zip(n_papers, entropies),
  key=operator.itemgetter(0), reverse=True))

In [ ]:
mask = ~np.isinf(entropies)
n_papers_mask = np.array(n_papers)[mask]
entropies_mask = entropies[mask]

In [ ]:
popt, pcov = curve_fit(func, n_papers_mask, entropies_mask)

In [ ]:
popt

In [ ]:
plt.scatter(n_papers_mask, entropies_mask)
plt.plot(np.array(n_papers), func(np.array(n_papers), *popt), 'r-',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))


In [ ]:
dist = func(np.array(n_papers), *popt)

In [ ]:
plt.scatter(np.log(n_papers), np.log(1 / (entropies - dist)))

In [ ]:
pid = 202008
papers = papers = openaire_publications_df[[True if str(pid) in s else False 
                                       for s in openaire_publications_df['ec_project_codes']]]
papers = papers[~pd.isnull(papers)]

In [ ]:
papers

In [ ]:
papers['language'].value_counts().index[0]

In [ ]:
cordis_projects_df[cordis_projects_df['id'] == pid]

In [ ]:
cordis_projects_df['oa_abstract_entropy_mean'] = entropies
cordis_projects_df['oa_n_papers'] = n_papers
cordis_projects_df['oa_abstract_lengths'] = lengths

In [ ]:
from datetime import datetime

In [ ]:
closed_projects = cordis_projects_df[cordis_projects_df['endDate'] < datetime.today()]

In [ ]:
plt.hist(entropies, bins=50)
plt.show()

In [ ]:
from eu_funding.data.parse import concat_entities

In [ ]:
cordis_projects_df

In [ ]:
cordis_topics_ref_df.head()

In [ ]:
cordis_projects_df.topics.value_counts()